<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/english/python/effective_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study

Using the example of the algorithm for finding prime numbers, we present the design of the efficient algorithm and the creation of the Pythonic code.

## First naive algoritm

A prime number is a natural number that has two divisors (itself and one). The smallest prime number is two. We can decide whether a number is prime by trying to find the remainder of the division with smaller numbers. Is it necessary to proceed with the examination of divisibility up to n-1 if n is the examined number? It is not worth checking for numbers larger than the square root of the number, since, for example, in the case of 24, after finding the divisor of four, it does not matter if we also find the divisor pair (6) belonging to four. This might look like this in Python:

In [1]:
max_num = 500001    # largest number + 1 to search primes

In [6]:
"""
    naive algorith to find prime numbers
    version 1.0
"""

import math
import time

start_time = time.time()
prims = []                   # list of prims
for p in range(2, max_num):   # find prims up to max_num
    prime = True
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0:     # remainder of division is zero
            prime = False        # it is not a prime
    if prime:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 17.416287 seconds


The efficiency of the algorithm is measured by the running time of it. On today's computers, several applications and services are always running in parallel, so a single time measurement does not always give an average result. It is advised to run it several times to find the average running time.

## First improvement

In the case of 105 in the above algorithm, the examination of the divisors takes up to 11 (square root of 105 + 1), however, after finding the divisor of 3, it is unnecessary to continue the inner cycle, it is already a decided non-prime number. Let's modify the algorithm so that after finding the first divisor, it exits the inner loop (break statement).

In [5]:
"""
    naive algorith to find prime numbers
    version 1.1
"""

import math
import time

start_time = time.time()
prims = []                   # list of prims
for p in range(2, max_num):   # find prims up to 50000
    prime = True
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0: # remainder of division is zero
            prime = False    # it is not a prime
            break            # divider found no need to continue
    if prime:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 2.080281 seconds


The first version runs for over 15 seconds. The second version takes less than 2 seconds. In the case of nested loops, shortening the run of the inner loop by the break leads to a large increase in efficiency.

## Let's make the code Pythonic

We mentioned in the introduction that our goal is not only efficiency, but also the development of Pythonic code. In the Python language, we can also assign an else clausule to the loop, which is executed if we have not exited the execution of the loop with a break command. By using this, we can make our code shorter and perhaps easier to read. It becomes unnecessary to use the prime logical variable.

In [7]:
"""
    naive algorith to find prime numbers
    version 1.2
"""

import math
import time
start_time = time.time()
prims = []                   # list of prims
for p in range(2, max_num):  # find prims up to max_num
    for divider in range(2, int(math.sqrt(p))+1):
        if p % divider == 0: # remainder of division is zero
            break            # divider found no need to continue
    else:
        prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 2.051309 seconds


With this modification, our code did not become more efficient, but the code with fewer instructions is more beneficial.

All numbers can be broken down into a product of prime numbers. Thus, it is sufficient to perform the divisibility test on the previously found prime numbers. Let's modify our algorithm.

In [9]:
"""
        naive algorith to find prime numbers
        version 1.3
"""

import math
import time

start_time = time.time()
prims = []                   # list of prims
for p in range(2, max_num):  # find prims up to max_num
        maxp = int(math.sqrt(p))+1
        for divider in prims:    # enough to check prims!
                if p % divider == 0: # remainder of division is zero
                        break            # divider found no need to continue
                if maxp < divider:
                        prims.append(p)
                        break
        else:
                prims.append(p)      # store prime number
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.714117 seconds


## A more efficient algorithm

In the previous versions, we modified the code for efficiency while keeping our original idea to check reminders after divisions. Maybe we can get a more effective solution by reevaluating our original idea? Even Erasthotenes managed to do this with the invention of the Erasthotenes sieve. The basic idea of this is not to find the primes by dividing the individual numbers, but to create a series of natural numbers and remove the multiples of the individual numbers from it. It might look something like this:

In [13]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.0
"""

import math
import time

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    numbers[j+j::j] = [0 for k in numbers[j+j::j]] # use sieve

prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.158309 seconds


We used list comprehension in the code. This is faster than generating the list with a for loop.

The 

> [0 for k in numbers[j+j::j]]

row produces a list containing zeros, the length of which corresponds to the number of multiples of the value j. By assigning the value, we reset the multiples of the value j to zero in the list of numbers. Couldn't you have simply written the following statement?

> numbers[j+j::j] = 0

Unfortunately, this does not work, we cannot assign a scalar to a part of a list, but [0] does not work on the right side of the assignment either, because it would also only work for a continuous part of the original list.

This version generates prime numbers up to half a million in 2 tenths of a second. Compared to our first algorithm, we achieved a hundredfold speedup.

## Can we speed it up?

Let's analyze our code a bit. The loop variable j takes the values 2, 3, 4, ... during the run, so first we reset all even numbers from 4, then every third number from 6, then every fourth from 8. Let's stop here for a moment! Why do we zero the numbers divisible by four? We already set them to zero because they are divisible by two. The situation is similar, for example, with numbers divisible by nine, they have already been set to zero because they are divisible by three. It is not necessary to reset the elements to zero for every j, this is only necessary if the j-th element has not yet been zeroed. We can do this with an additional condition, which makes the code longer, but more efficient.

In [14]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.1
"""

import math
import time

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = [0 for k in numbers[j+j::j]] # use sieve

prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.103916 seconds


The efficiency-enhancing effect of this modification is less apparent when running up to half a million. This is also because the running time of our algorithm does not increase linearly as the maximum prime number increases.

List comprehension is a more efficient way to generate lists than a for loop. However, in our case, all elements of the generated list are null. The list interpretation is used to set the length of the list. However, there is a simpler (Pythonic) solution for this. The

> [0] * 5

statement results in a list of zeros of length five. Let's see if such a transformation increases efficiency!

In [16]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.2
"""

import math
import time

start_time = time.time()
numbers = list(range(max_num))     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = [0] * len(numbers[j+j::j]) # use sieve
prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.048999 seconds


With this modification, finding prime numbers up to five million takes less than 5 hundreds of a second.

## The numpy library can speed things up a bit

The numpy Python module provides programmers with ready-made solutions for many mathematical problems. We use numpy arrays and assigning values to multiple array elements.

In [17]:
"""
    Sieve of Erasthotenes prim algorithm
    version 2.3
"""

import math
import time
import sys
import numpy as np

start_time = time.time()
numbers = np.arange(max_num)     # list of natural numbers to check
for j in range(2, int(math.sqrt(max_num))):
    if numbers[j]:
        numbers[j+j::j] = 0 # use sieve
prims = sorted(list(set(numbers) - set([0, 1]))) # remove zeros from list
print('ready')
print('%d prims in %f seconds' % (len(prims), time.time() - start_time))

ready
41538 prims in 0.043427 seconds


Apart from importing the numpy module, only two lines were changed. During the generation of the numbers, we create a numpy array with the `arange` function. The speed-up is the second modification, in order to zero the elements, we do not need to create a list of as many zero elements as we want to zero. With this, we can achieve an additional speedup of around 10%, of course, here we did not include the time to load the numpy module.

# Task

Try to run the code with larger limit and analyze the elapsed time of the different versions.